In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from requests.adapters import HTTPAdapter, Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

import pickle
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin, urlunparse
import re

## Primero seleccionamos todos los links de las paginas base de El Colombiano

In [2]:

SOCIAL_HOSTS = {"facebook.com","m.facebook.com","web.facebook.com",
    "twitter.com","x.com","t.co",
    "linkedin.com","lnkd.in",
    "api.whatsapp.com","wa.me","web.whatsapp.com",
    "instagram.com","youtube.com","youtu.be","tiktok.com",
    "telegram.me","t.me","pinterest.com"}


BAD_SUBSTR = ("share", "sharer.php", "dialog/send", "utm_", "redirect_uri=")


BAD_PATH_PREFIX = ( "/multimedia/imagenes/", "/multimedia/videos/", "/buscador/",
    "/servicios/", "/suscribete/", "/suscripciones/",)


GOOD_SECTIONS = {"colombia","medellin","antioquia","deportes","internacional",
    "negocios","entretenimiento","opinion","cultura","tendencias",
    "tecnologia","economia","construccion","metro","medio-ambiente",
    "redes-sociales",
    "politica","salud","educacion","paz-y-derechos-humanos",
    "empresas","finanzas","agro",
    "futbol","formula-1","atletico-nacional","independiente-medellin",
    "seguridad","movilidad","obras",
    "cine","literatura","musica","mascotas",
    "ciencia","gadgets","videojuegos","aplicaciones",
    "motores","turismo","moda","farandula","television"}


ART_ID_REGEX = re.compile(r"-[A-Z0-9]{6,}$")

## Todo el texto en un unico formato
def _normalize(base_url: str, href: str):
    absu = urljoin(base_url, href)
    p = urlparse(absu)
    p = p._replace(query="", fragment="")
    return urlunparse(p)

# Identificar si el link es una noticia del colombiano o es externo
def _is_internal_or_elcolombiano(href: str):
    if not href or href.startswith(("#", "javascript:", "mailto:", "tel:")):
        return False
    
    if href.startswith("/"):
        return True
    
    host = urlparse(href).netloc.lower()

    if any(host.endswith(s) for s in SOCIAL_HOSTS):
        return False
    
    return host.endswith("elcolombiano.com")

# Identificar si el link es un articulo o otra cosa como un podcast ... 
def _looks_like_article(url: str, be_strict: bool) :
    p = urlparse(url)
    path = p.path

    if any(path.startswith(bad) for bad in BAD_PATH_PREFIX):
        return False
    if any(s in url for s in BAD_SUBSTR):
        return False
    
    if be_strict:
        if ART_ID_REGEX.search(path):
            return True
        if path.startswith("/opinion/columnistas/") and path.count("/") >= 3:
            return True
        return False
    parts = [p for p in path.split("/") if p]

    if len(parts) >= 2:
        if parts[0] not in GOOD_SECTIONS:
            print("Nueva sección detectada:", parts[0])
        return True
    
    return False


# Traer todos los links de noticias de la pagina header que le pasemos
def get_all_news_links(page_url: str, strict: bool = True):

    r = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"}, timeout=25)
    r.raise_for_status()
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text, "html.parser")

    links = set()
    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        if not _is_internal_or_elcolombiano(href):
            continue
        url = _normalize(page_url, href)
        if _looks_like_article(url, be_strict=strict):
            links.add(url)

    return sorted(links)

In [3]:
links_colombiano = [
                    "https://www.elcolombiano.com/medellin" , 
                    "https://www.elcolombiano.com/colombia" , 
                    "https://www.elcolombiano.com/negocios" , 
                    "https://www.elcolombiano.com/deportes" , 
                    "https://www.elcolombiano.com/antioquia" , 
                    "https://www.elcolombiano.com/cultura" ,
                    "https://www.elcolombiano.com/tendencias" , 
                    "https://www.elcolombiano.com/tecnologia" , 
                    "https://www.elcolombiano.com/entretenimiento" , 
                    'https://www.elcolombiano.com/colombia/politica' , 
                    'https://www.elcolombiano.com/colombia/salud' , 
                    'https://www.elcolombiano.com/colombia/educacion' , 
                    'https://www.elcolombiano.com/colombia/paz-y-derechos-humanos' ,
                    'https://www.elcolombiano.com/negocios/empresas' , 
                    'https://www.elcolombiano.com/negocios/finanzas' , 
                    'https://www.elcolombiano.com/negocios/agro' , 
                    'https://www.elcolombiano.com/deportes/futbol',
                    'https://www.elcolombiano.com/deportes/formula-1' ,
                    'https://www.elcolombiano.com/deportes/atletico-nacional' , 
                    'https://www.elcolombiano.com/deportes/independiente-medellin' , 
                    'https://www.elcolombiano.com/antioquia/seguridad' , 
                    'https://www.elcolombiano.com/antioquia/movilidad' , 
                    'https://www.elcolombiano.com/antioquia/obras' , 
                    'https://www.elcolombiano.com/cultura/cine' , 
                    'https://www.elcolombiano.com/cultura/literatura' , 
                    'https://www.elcolombiano.com/cultura/musica' , 
                    'https://www.elcolombiano.com/cultura/mascotas' , 
                    'https://www.elcolombiano.com/tecnologia/ciencia' , 
                    'https://www.elcolombiano.com/entretenimiento/motores' , 
                    'https://www.elcolombiano.com/entretenimiento/turismo' , 
                    'https://www.elcolombiano.com/medio-ambiente' , 
                    'https://www.elcolombiano.com/tecnologia/gadgets' , 
                    'https://www.elcolombiano.com/tecnologia/videojuegos' , 
                    'https://www.elcolombiano.com/tecnologia/aplicaciones' , 
                    'https://www.elcolombiano.com/redes-sociales/trending-topic' , 
                    'https://www.elcolombiano.com/entretenimiento/moda' , 
                    'https://www.elcolombiano.com/entretenimiento/farandula' , 
                    'https://www.elcolombiano.com/entretenimiento/television']


links_tuplas = [(link, link.rstrip('/').split('/')[-1]) for link in links_colombiano]
links_tuplas[:4]

[('https://www.elcolombiano.com/medellin', 'medellin'),
 ('https://www.elcolombiano.com/colombia', 'colombia'),
 ('https://www.elcolombiano.com/negocios', 'negocios'),
 ('https://www.elcolombiano.com/deportes', 'deportes')]

In [4]:
links_totales = []
for i in links_tuplas: 
    temp_links = get_all_news_links(i[0])
    temp_tuplas = [(noticia , i[1]) for noticia in temp_links]
    links_totales.extend(temp_tuplas)

print(links_totales[:10])
print(len(links_totales))

[('https://www.elcolombiano.com/medellin/alcaldia-de-medellin-cerro-dos-glampings-en-san-cristobal-IG28959364', 'medellin'), ('https://www.elcolombiano.com/medellin/buggies-parque-norte-atraccion-precios-HH28949608', 'medellin'), ('https://www.elcolombiano.com/medellin/capturado-en-medellin-bernabe-vasquez-por-trafico-opioides-LF28968892', 'medellin'), ('https://www.elcolombiano.com/medellin/el-era-fernando-el-joven-que-murio-tras-ser-apunalado-en-unos-15-FG28955312', 'medellin'), ('https://www.elcolombiano.com/medellin/el-ferrari-y-el-rolex-de-miguel-quintero-hermano-de-daniel-quintero-con-presunta-corrupcion-OG28957485', 'medellin'), ('https://www.elcolombiano.com/medellin/fiscalia-investiga-daniel-quintero-corrupcion-afinia-JB28920009', 'medellin'), ('https://www.elcolombiano.com/medellin/fotos-a-los-mapaches-pumas-y-perros-zorro-que-hay-en-los-bosques-de-medellin-FG28959518', 'medellin'), ('https://www.elcolombiano.com/medellin/loma-de-el-tesoro-abren-licitacion-de-obra-por-15-000-

## Funciones para scrapear noticias en general

In [5]:
# Limpieza básica 
def _clean(t) :
    if t is None:
        return None
    t = " ".join(t.split())
    return t or None

# Para unir los parrafos por separado de la pagina
def _join_paragraphs(parrafos):
    pars = [_clean(p) for p in parrafos if _clean(p)]
    filt = []
    for p in pars:
        if p and not re.search(r"Lea también|Puede interesarle|Siga leyendo|Podcast|Video", p, re.I):
            filt.append(p)
    return " ".join(filt) if filt else None


# Scraper noticia 
def scrape_noticia(url, session = None, timeout=(5, 40)):

    sess = session or requests.Session()
    r = sess.get(url, timeout=timeout)
    r.raise_for_status()
    r.encoding = "utf-8"

    soup = BeautifulSoup(r.text, "lxml")

    # Hora 
    hora = None
    hora_el = soup.select_one(".hora-noticia, time.hora-noticia, .fecha, time[datetime]")

    if hora_el:
        hora = _clean(hora_el.get_text(" ", strip=True))


    if not hora:
        for s in soup.select('script[type="application/ld+json"]'):
            try:
                data = json.loads(s.string or "")
                items = data if isinstance(data, list) else [data]
                for it in items:
                    if isinstance(it, dict):
                        if it.get("@type") in ("NewsArticle", "Article"):
                            hora = _clean(it.get("datePublished") or it.get("dateModified"))
                            if hora:
                                break
                if hora:
                    break
            except Exception:
                pass

    # Autor 
    autor = None
    autor_el = soup.select_one(".div_author_r_texto_, .autor, .author, .byline, .firma")
    if autor_el:
        raw_text = autor_el.get_text(" ", strip=True)
        candidatos = re.findall(r"[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s+[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)+", raw_text)
        autor = _clean(candidatos[0]) if candidatos else _clean(raw_text.split("hace")[0])

    if not autor:
        meta_author = soup.find("meta", attrs={"name": "author"}) or soup.find("meta", attrs={"property": "article:author"})
        if meta_author and meta_author.get("content"):
            autor = _clean(meta_author["content"])

    # Título 
    titulo = None
    tit_el = soup.select_one(".priority-content, h1.titulo, h1.title, h1.headline, h1")
    if tit_el:
        titulo = _clean(tit_el.get_text(" ", strip=True))
    if not titulo:
        og_title = soup.find("meta", attrs={"property": "og:title"})
        if og_title and og_title.get("content"):
            titulo = _clean(og_title["content"])

    # Cuerpo 
    cuerpo = None
    cuerpo_div = soup.select_one("div.block-text, article .texto, article .cuerpo, article .content, .article-body, .texto-noticia")
    if cuerpo_div:
        pars = [p.get_text(" ", strip=True) for p in cuerpo_div.find_all("p")]
        cuerpo = _join_paragraphs(pars)
    if not cuerpo:
        pars = [p.get_text(" ", strip=True) for p in soup.select("article p")]
        cuerpo = _join_paragraphs(pars)

    return {"hora": hora, "titulo": titulo, "autor": autor, "cuerpo": cuerpo, "url": url}



In [6]:
scrape_noticia('https://www.elcolombiano.com/deportes/futbol/alejandro-restrepo-rueda-de-prensa-dim-clasificado-cuartos-de-final-copa-betplay-HF28963361')

{'hora': 'hace 6 horas',
 'titulo': 'Alejandro Restrepo tras clasificación del DIM en Copa Betplay: “Estos partidos se ganan con actitud y convicción”',
 'autor': 'Sara Gil Martínez',
 'cuerpo': 'El Deportivo Independiente Medellín vivió una noche de emociones en el Atanasio Girardot. Con goles de Francisco Fydriszewski al minuto 33 y de Léider Berrío en el 90’, venció 2-1 a Fortaleza y selló su clasificación a los cuartos de final de la Copa Betplay. En la rueda de prensa posterior al compromiso, el técnico Alejandro Restrepo reconoció la dificultad del rival y destacó la forma en que su equipo supo interpretar el partido. “ Fortaleza tenía 12 o 13 partidos sin perder, eso habla de la calidad de equipo que es. Fueron dos partidos durísimos, casi un juego de ajedrez. Más allá de la victoria, hay un sentimiento como entrenador de ver propuestas innovadoras en el fútbol colombiano. Afortunado de tener este grupo que entendió el partido y salió a ganarlo ”, expresó. El gol de la clasifica

## Funcion para scrapear una noticia que escribio un columnista del colombiano

In [34]:
columnistas= ['https://www.elcolombiano.com/cronologia/noticias/meta/diego-santos',
'https://www.elcolombiano.com/cronologia/noticias/meta/luis-diego-monsalve',
'https://www.elcolombiano.com/cronologia/noticias/meta/mauricio-perfetti-del-corral',
'https://www.elcolombiano.com/cronologia/noticias/meta/oscar-dominguez-giraldo',
'https://www.elcolombiano.com/cronologia/noticias/meta/rodrigo-botero-montoya' ,
'https://www.elcolombiano.com/cronologia/noticias/meta/isabel-gutierrez-ramirez',
'https://www.elcolombiano.com/cronologia/noticias/meta/humberto-montero',
'https://www.elcolombiano.com/cronologia/noticias/meta/daniel-duque',
'https://www.elcolombiano.com/cronologia/noticias/meta/alberto-velasquez-martinez',
'https://www.elcolombiano.com/cronologia/noticias/meta/johel-moreno-s-',
'https://www.elcolombiano.com/cronologia/noticias/meta/paola-holguin',
'https://www.elcolombiano.com/cronologia/noticias/meta/david-e-santos-gomez']

# Limpiar el texto
def _clean(t):
    if t is None:
        return None
    t = " ".join(t.split())
    return t or None

# Unir los parrafos
def _join_paragraphs(parrafos):
    parrafos = [_clean(p) for p in parrafos if _clean(p)]
    return " ".join(parrafos) if parrafos else None


# Scrapper para los articulos escritos por un columnista
def scrape_columnista(url, session = None, timeout=(5, 40)):

    sess = session or requests.Session()
    r = sess.get(url, timeout=timeout)
    r.raise_for_status()
    r.encoding = "utf-8"

    soup = BeautifulSoup(r.text, "lxml")

    # Hora 
    hora_el = soup.select_one(".hora-noticia, time.hora-noticia, .fecha, time[datetime]")
    hora = _clean(hora_el.get_text(strip=True)) if hora_el else None

    # Autor
    autor_el = soup.select_one(".texto-columnista p, .autor, .byline, .firma, .author , .nombre-columnista")
    autor = _clean(autor_el.get_text(strip=True)) if autor_el else None

    # Título 
    tit_el = soup.select_one(".priority-content, h1.titulo, h1.title, h1")
    titulo = _clean(tit_el.get_text(strip=True)) if tit_el else None

    # principal
    cuerpo_div = soup.select_one("div.texto-columnista div.text, .texto-columnista .text, article .texto, article .cuerpo")
    if cuerpo_div:
        pars = [p.get_text(" ", strip=True) for p in cuerpo_div.find_all("p")]
        cuerpo = _join_paragraphs(pars)
    else:
        # fallback: buscar párrafos del artículo
        pars = [p.get_text(" ", strip=True) for p in soup.select("article p")]
        cuerpo = _join_paragraphs(pars)

    return {"hora": hora,
        "titulo": titulo,"autor": autor, "cuerpo": cuerpo,
        'link': url}


scrape_columnista('https://www.elcolombiano.com/opinion/columnistas/de-sainete-a-tragicomedia-DJ19457774')


{'hora': None,
 'titulo': 'De sainete a tragicomedia',
 'autor': 'Alberto Velásquez Martínez',
 'cuerpo': 'Hasta el pasado lunes – a la hora de enviar esta columna – la fecha para poner en marcha las dos primeras turbinas de Hidroituango era incierta. Seguía la escenificación del melodrama que ha envuelto toda la obra. Su calvario comenzó desde que a la Montaña se le perforó. No quería tubos ni turbinas en sus entrañas. Tuvo momentos de calma que suponían que había asimilado esa penetración de cuerpo extraño en su panza. Los detectores, no de mentiras – pues estos son reservados para las autoridades que han manipulado las escenas del drama – registraban vibraciones de una geología que no se resignaba a estrujones. Desde el primer barretazo para la construcción de la presa para contener el caudaloso y encañonado río Cauca para sacarle kilovatios, comenzó el debate. Se rodeó de cábalas, demagogia y datos que se quisieron meter dentro de expedientes de derecho administrativo y penal. Por 

## Paralelizando las descargas y usando conexiones reutilizables scrappeemos todas las noticias de los links que optuvimos de las paginas estaticas, de una forma eficiente y sin que el servidor nos bloquee

In [10]:
# Crear sesion para hacer seguro el scrapping
def build_session():
    s = requests.Session()

    retries = Retry(total=3,             
        backoff_factor=0.5, status_forcelist=(429, 500, 502, 503, 504),allowed_methods=("GET", "HEAD"))
    
    adapter = HTTPAdapter(max_retries=retries,
        pool_connections=100, pool_maxsize=100)
    
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    s.headers.update({"User-Agent": "Mozilla/5.0"})
    s.timeout = (5, 45)  
    return s

# Scrapear noticia por link de forma segura
def _safe_scrape_any(url, categoria, session, columnistas_urls) :

    try:
        is_col = ("columnistas" in url.lower()) or (columnistas_urls and url in columnistas_urls)
        if is_col:
            item = scrape_columnista(url, session=session)  

        else:
            item = scrape_noticia(url, session=session)

        if not item:
            return None
        item["categoria"] = categoria
    
        return item
    
    except requests.RequestException as e:
        print(f"[WARN] request error en {url}: {e}")
        return None
    
    except Exception as e:
        print(f"[WARN] parse error en {url}: {e}")
        return None
    

# Scrapear todas las noticias NO ESCRITAS por columnistas
def scrapping_all(l, columnistas = None):

    session = build_session()
    out = []
    total = len(l)
    for i, (url, cat) in enumerate(l, start=1):
        item = _safe_scrape_any(url, cat, session=session, columnistas_urls=columnistas)

        if item:
            out.append(item)

        if 50 and (i % 50 == 0):
            print(f"[INFO] Procesadas {i}/{total}")

    return out


noticias1 = scrapping_all(links_totales)
noticias1[:10]

[INFO] Procesadas 50/580
[INFO] Procesadas 100/580
[INFO] Procesadas 150/580
[INFO] Procesadas 200/580
[INFO] Procesadas 250/580
[INFO] Procesadas 300/580
[INFO] Procesadas 350/580
[INFO] Procesadas 400/580
[INFO] Procesadas 450/580
[INFO] Procesadas 500/580
[INFO] Procesadas 550/580


[{'hora': 'hace 9 horas',
  'titulo': 'Alcaldía de Medellín cerró dos glampings en San Cristóbal; ¿por qué?',
  'autor': 'El Colombiano',
  'cuerpo': 'La Secretaría de Gestión y Control Territorial de Medellín, la Corregiduría de San Cristóbal y la Policía Nacional realizaron esta semana una serie de operativos de vigilancia y control en zona rural del noroccidente de Medellín en los que se encontraron con que algunos glampings tenían serias irregularidades. Hay que recordar que este tipo de hospedajes ofrecen generalmente servicios de alojamientos de lujo que incluyen experiencias inmersivas en la naturaleza, por lo que suelen estar en zonas boscosas. Puede leer: ¡Qué guachafita! Turistas destrozaron un glamping en Guatapé y huyeron En el sitio, las autoridades inspeccionaron varias construcciones tipo glamping , que dejan como resultado dos suspensiones de obras que no contaban con licencia de construcción ni requisitos para operar como alojamientos, así como seis informes técnicos. 

## Genrar todos los links de todas las notiicas que hayan publicado los 12 columnistas 

In [15]:

BAD_SUBSTR = ("share", "sharer.php", "dialog/send", "utm_", "redirect_uri=")

# Verificar si estamos trayendo una noticia de algun columnista o otra cosa 
def _is_article(url: str, base_host: str):
    p = urlparse(url)
    host = p.netloc.lower().removeprefix("www.")

    if not (host == base_host or host.endswith("." + base_host)):
        return False
    if any(s in url for s in BAD_SUBSTR):
        return False
    return True


# Normalizar los links 
def _normalize(href: str, listing_url: str):
    absu = urljoin(listing_url, href)
    p = urlparse(absu)
    netloc = p.netloc.lower().removeprefix("www.")
    p = p._replace(scheme="https", netloc=netloc, query="", fragment="")
    if p.path != "/" and p.path.endswith("/"):
        p = p._replace(path=p.path.rstrip("/"))
    return urlunparse(p)


# Con esta funcion extrameos todos los links de una pagina de columnista 
# Esto trae los links de todos los articulos de un columnista 
def get_listing_article_links_from_html(listing_url: str, html: str):

    base_host = urlparse(listing_url).netloc.lower().removeprefix("www.")
    soup = BeautifulSoup(html, "html.parser")
    links = set()

    containers = soup.select("div.ec-teaser-noticia-seccion-metadatos, "
        "div.container-noticia-seccion-metadatos, "
        "div.text_container_noticia_metadato")

    for box in containers:
        a = None
        for cand in box.select("a[href]"):
            if cand.select_one("span.priority-content, h3, .title__link"):
                a = cand
                break
        if not a:
            a = box.find("a", href=True)
        if not a:
            continue
        url = _normalize(a["href"].strip(), listing_url)
        if _is_article(url, base_host):
            links.add(url)

    for a in soup.select("a[href]"):
        href = a["href"].strip()
        if not href or href.startswith(("#", "javascript:", "mailto:", "tel:")):
            continue
        url = _normalize(href, listing_url)
        if _is_article(url, base_host) and a.select_one("span.priority-content, h3"):
            links.add(url)

    return sorted(links)


# Funcion que sirve para darle al boton de 'CARGAR MAS' en las paginas de los columnistas 
# y cuando no se puedan cargar mas noticias (no tiene mas ese columnista) se traen todos los 
# links de todas las noticias mediante la funcion de arriba
def get_links_after_clicks(listing_url: str, max_clicks: int = 80, wait_secs: int = 12):

    opts = webdriver.ChromeOptions()
    opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)

    try:
        driver.get(listing_url)
        wait = WebDriverWait(driver, wait_secs)

        def count_cards():
            return len(driver.find_elements(By.CSS_SELECTOR, "article"))

        clicks = 0
        
        while clicks < max_clicks:
            try:
                btn = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "div.more-button[id$='_myMoreButton']")))
            except Exception:
                break  

            before = count_cards()
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
            time.sleep(0.2)
            driver.execute_script("arguments[0].click();", btn)

            try:
                wait.until(EC.invisibility_of_element_located(
                    (By.CSS_SELECTOR, "div[id$='_loadingDiv']")))
            except Exception:
                pass

            try:
                wait.until(lambda d: count_cards() > before)
                clicks += 1
                time.sleep(0.3)
            except Exception:
                break

        html = driver.page_source
    finally:
        driver.quit()

    return get_listing_article_links_from_html(listing_url, html)

In [10]:

total_noticas_columnistas = []

for i in columnistas:
    links = get_links_after_clicks(i)
    print("Total:", len(links))
    total_noticas_columnistas.extend(links)

# Guardar los Links
with open("lista_columnistas.pkl", "wb") as f:
    pickle.dump(total_noticas_columnistas, f)


Total: 62
Total: 54
Total: 109
Total: 544
Total: 277
Total: 40
Total: 564
Total: 104
Total: 565
Total: 59
Total: 81
Total: 570


In [19]:
total_noticas_columnistas[:3]

['https://elcolombiano.com/opinion/columnistas/diego-santos-a-petro-y-al-pacto-historico-se-les-puede-ganar-pero-no-con-la-apuesta-actual-GB26365201',
 'https://elcolombiano.com/opinion/columnistas/diego-santos-alfredo-mondragon-alias-el-maton-BF26895136',
 'https://elcolombiano.com/opinion/columnistas/diego-santos-antioquia-en-sus-manos-estamos-GN26090402']

## Paralelizando las descargas y usando conexiones reutilizables scrappeemos todas las noticias de los columnistas, de una forma eficiente y sin que el servidor nos bloquee, usando los links generados anteriormente 

In [35]:
# Hacer seguro el scrapping de las noticas de columnistas
def _safe_scrape(url, session):
    try:

        return scrape_columnista(url, session=session)
    
    except requests.RequestException as e:
        print(f"[WARN] fallo en {url}: {e}")
        return None

# En paralelo y de forma segura extraemos toda la informacion de las noticias de todos 
# los links de todos los columnistas del colombiano
def scrapping_columnistas(urls, max_workers=12, sleep_jitter=0.0):

    session = build_session()
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = []
        for u in urls:
            if sleep_jitter:
                time.sleep(sleep_jitter)
            futures.append(ex.submit(_safe_scrape, u, session))

        for idx, fut in enumerate(as_completed(futures), start=1):
            item = fut.result()
            if item is not None:
                results.append(item)
                
            if idx % 100 == 0:
                print(f"[INFO] Procesadas {idx} noticias de {len(urls)}")

    return results

noticias_columnistas = scrapping_columnistas(total_noticas_columnistas)
noticias_columnistas[:3]

[INFO] Procesadas 100 noticias de 3029
[INFO] Procesadas 200 noticias de 3029
[INFO] Procesadas 300 noticias de 3029
[INFO] Procesadas 400 noticias de 3029
[INFO] Procesadas 500 noticias de 3029
[INFO] Procesadas 600 noticias de 3029
[INFO] Procesadas 700 noticias de 3029
[INFO] Procesadas 800 noticias de 3029
[INFO] Procesadas 900 noticias de 3029
[INFO] Procesadas 1000 noticias de 3029
[INFO] Procesadas 1100 noticias de 3029
[INFO] Procesadas 1200 noticias de 3029
[INFO] Procesadas 1300 noticias de 3029
[INFO] Procesadas 1400 noticias de 3029
[INFO] Procesadas 1500 noticias de 3029
[INFO] Procesadas 1600 noticias de 3029
[INFO] Procesadas 1700 noticias de 3029
[INFO] Procesadas 1800 noticias de 3029
[INFO] Procesadas 1900 noticias de 3029
[INFO] Procesadas 2000 noticias de 3029
[INFO] Procesadas 2100 noticias de 3029
[INFO] Procesadas 2200 noticias de 3029
[INFO] Procesadas 2300 noticias de 3029
[INFO] Procesadas 2400 noticias de 3029
[INFO] Procesadas 2500 noticias de 3029
[INFO] Pr

[{'hora': '17 de abril de 2025',
  'titulo': 'El “apartheid” de Israel contra los árabes',
  'autor': 'Por Diego Santos - @diegoasantos',
  'cuerpo': 'Detrás de mí hay dos mujeres árabes. Nos separa un cristal. Yo estoy adentro de la cafetería y ellas afuera, en una terraza cubierta por vidrios. ¡El apartheid israelí! ¡Las discriminan y humillan por ser árabes! Yo adentro cobijado por el calor interior y ellas afuera, congeladas del frío. Así relataría un progresista antisemita la escena que les estoy narrando. Pero préstenme cinco minutos para narrarles la experiencia real. Las mujeres árabes que estaban afuera se encontraban en la zona de fumadores, ambas gozando unos cigarrillos mientras se reían tomando café. Le pedí a mi acompañante que me tomara una foto con ellas de fondo. Lo hice. Una de ellas sonrío mirando a la cámara, mientras la otra se giró discretamente para no salir en la foto. La radiografía de la relación entre Israel y los árabes hay que vivirla en Israel, y no a trav

## Crear los DataFrame completos


In [43]:
def preparar_df(lista):

    hoy = '4 de Septiembre de 2025'
    df = pd.DataFrame(lista)
    df = df.replace("", np.nan)
    df_limpio = df.dropna(subset=["titulo", "cuerpo"])

    pat = r'^\s*(?:\d+\s*y\s*\d+\s*|no\s+){4,}\s*'  
    df_limpio["cuerpo"] = df_limpio["cuerpo"].str.replace(pat, "", regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
    df_limpio.loc[:, "hora"] = df_limpio["hora"].apply(lambda x: hoy if "hace" in str(x) else x)

    return df_limpio


noticias_columnistas = preparar_df(noticias_columnistas)
noticias_columnistas


,hora,titulo,autor,cuerpo,link
0,17 de abril de 2025,El “apartheid” de Israel contra los árabes,Por Diego Santos - @diegoasantos,Detrás de mí hay dos mujeres árabes. Nos separ...,https://elcolombiano.com/opinion/columnistas/d...
1,16 de enero de 2025,A Petro y al Pacto Histórico se les puede gana...,Por Diego Santos - @diegoasantos,La mayoría de los colombianos queremos que en ...,https://elcolombiano.com/opinion/columnistas/d...
2,10 de abril de 2025,Antioquia tiene un gran futuro representativo,Por Diego Santos - @diegoasantos,Si los antioqueños tienen algún tipo de preocu...,https://elcolombiano.com/opinion/columnistas/d...
3,31 de octubre de 2024,El acto legislativo de Juan Fernando Cristo no...,Por Diego Santos - @diegoasantos,"Hábil y oportunista como es, el ministro del I...",https://elcolombiano.com/opinion/columnistas/d...
4,23 de enero de 2025,¿Cuál es la Colombia que queremos?,Por Diego Santos - @diegoasantos,Nos vamos acercando a las elecciones de 2026 y...,https://elcolombiano.com/opinion/columnistas/d...
...,...,...,...,...,...
3024,None,Votos y violencia,David E. Santos Gómez,"El 6 de septiembre de 2018, solo unos meses an...",https://elcolombiano.com/opinion/columnistas/v...
3025,None,¿Y si fuera musulmán?,David E. Santos Gómez,Pocos eventos despiertan tanto el morbo period...,https://elcolombiano.com/opinion/columnistas/y...
3026,None,¿Y el Presidente?,David E. Santos Gómez,A Juan Manuel Santos le queda menos de un mes ...,https://elcolombiano.com/opinion/columnistas/y...
3027,None,¿Y Venezuela qué?,David E. Santos Gómez,Se ha anunciado tantas veces la aparición del ...,https://elcolombiano.com/opinion/columnistas/y...


In [44]:
noticias_estatico = preparar_df(noticias1)
noticias_estatico

,hora,titulo,autor,cuerpo,url,categoria
0,4 de Septiembre de 2025,Alcaldía de Medellín cerró dos glampings en Sa...,El Colombiano,La Secretaría de Gestión y Control Territorial...,https://www.elcolombiano.com/medellin/alcaldia...,medellin
1,03 de septiembre de 2025,"En el Parque Norte, en Medellín, ya es posible...",El Colombiano,Medellín sigue en su esfuerzo por recuperar su...,https://www.elcolombiano.com/medellin/buggies-...,medellin
2,4 de Septiembre de 2025,Capturaron en Medellín a un peligroso trafican...,El Colombiano,En un operativo conjunto entre la Fiscalía Gen...,https://www.elcolombiano.com/medellin/capturad...,medellin
3,03 de septiembre de 2025,"Él era Fernando, el joven que murió tras ser a...",03 de septiembre de 2025 bookmark Guardar,"El próximo 6 de septiembre, la Nueva Villa de ...",https://www.elcolombiano.com/medellin/el-era-f...,medellin
4,4 de Septiembre de 2025,El Ferrari y el Rolex con los que presumía Mig...,None,"Miguel Andrés Quintero Calle ríe a carcajadas,...",https://www.elcolombiano.com/medellin/el-ferra...,medellin
...,...,...,...,...,...,...
575,25 de agosto de 2025,El público será jurado en la nueva versión de ...,Colprensa 25 de agosto de 2025 bookmark Guardar,El Canal RCN adquirió los derechos de Miss Uni...,https://www.elcolombiano.com/entretenimiento/r...,television
576,28 de agosto de 2025,¿Qué pasó con el pan? Polémica en el Desafío d...,El Colombiano,En el último capítulo del reality fueron varia...,https://www.elcolombiano.com/entretenimiento/t...,television
577,01 de septiembre de 2025,Estas son las series y películas que llegarán ...,El Colombiano,Si ya no tiene qué ver en plataformas de strea...,https://www.elcolombiano.com/entretenimiento/t...,television
578,19 de agosto de 2025,“Es una serie que exalta la latinidad”: el act...,Claudia Arango Holguín Tendencias,"Si no ha visto Acapulco , la comedia que ya ll...",https://www.elcolombiano.com/entretenimiento/t...,television


In [45]:
noticias_columnistas.to_csv('ElColombiano_Columnistas.csv')

In [46]:
noticias_estatico.to_csv('ElColombiano_Estatico.csv')